# Semantic Memory in Customer Support: Insurance Domain Demo

This notebook demonstrates how **Semantic Memory** can be used in customer support agents to learn and apply domain knowledge across different customer interactions.

## Core Concept: Knowledge Emerges from Conversations

Unlike traditional knowledge bases that require manual curation, semantic memory automatically extracts and stores general concepts, principles, and domain expertise from real customer conversations. When a new customer asks about a similar topic, the agent can apply this learned knowledge to provide accurate, consistent responses.

**Key Benefits:**
- **Cross-Customer Learning**: Knowledge from one interaction helps all future customers
- **Consistent Expertise**: All agents apply the same domain principles  
- **Automatic Knowledge Building**: No manual knowledge base maintenance required
- **Conceptual Understanding**: Agents understand underlying principles, not just specific cases

This demo shows how an insurance support agent learns tree damage liability principles from one conversation and applies them to help different customers with similar scenarios.

## Dependencies and Setup

We'll use several key libraries to build our semantic memory system:

- **Mem0**: The core memory platform that automatically extracts and stores semantic knowledge from conversations
- **LangGraph**: Creates stateful workflows that can retrieve, process, and store memory
- **LangChain AWS**: Provides the LLM interface for generating responses


The beauty of semantic memory is that it works automatically - Mem0 analyzes conversations and extracts general principles without requiring manual knowledge base creation.

In [ ]:
!pip install langgraph langchain-aws mem0ai

In [16]:
import asyncio
from typing import Dict, List, Any, TypedDict
import json
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_aws import ChatBedrockConverse
from mem0 import MemoryClient
import os

## Memory Client Initialization

The `MemoryClient` is the gateway to our semantic memory system. It handles:

- **Automatic Knowledge Extraction**: Analyzes conversations and identifies key concepts, patterns, and domain principles
- **Cross-Agent Memory Sharing**: Uses `agent_id` to create shared knowledge pools across all support interactions
- **Intelligent Search**: Retrieves relevant semantic knowledge based on query similarity and context

By using a shared `agent_id` (like "insurance_semantic_agent"), all conversations contribute to a collective knowledge base that every support interaction can benefit from.

In [3]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("MEM0_API_KEY")

In [4]:
import os
from mem0 import MemoryClient

memory_client = MemoryClient()

## Language Model Configuration

We configure Claude 3.5 Haiku through AWS Bedrock for our insurance support agent. This LLM will be used to analyze customer queries and generate responses based on the combined knowledge from semantic memory and the current interaction.

In [6]:
llm = ChatBedrockConverse(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0.1,
)

## State Management

The `SupportState` represents our conversation's working memory - the information that flows between different processing steps:

- **customer_id**: Identifies the current customer (though semantic memory is shared across all customers)
- **customer_query**: The current question or problem to solve
- **semantic_knowledge**: Retrieved domain expertise from past conversations 
- **support_response**: The final AI-generated response

This simplified state focuses purely on semantic memory - demonstrating how general domain knowledge enhances every customer interaction, regardless of their personal history.

In [7]:
class SupportState(TypedDict):
    """Simplified state for semantic memory demo"""
    customer_id: str
    customer_query: str
    semantic_knowledge: List[Dict[str, Any]]
    support_response: str

## Semantic Knowledge Retrieval

This is where the magic happens - we search our semantic memory for relevant domain knowledge:

**Key Insight**: Using a shared `agent_id` means all insurance support conversations contribute to one collective knowledge base. When Customer A teaches the system about "Acts of God" in tree damage, Customer B automatically benefits from this knowledge.

The retrieved memories aren't exact conversation matches - they're extracted principles and concepts that apply broadly across similar situations.

In [8]:
async def retrieve_semantic_knowledge(state: SupportState) -> SupportState:
    """Retrieve relevant semantic (general) knowledge about insurance concepts"""
    
    # Search for general insurance knowledge related to the query
    semantic_memories = memory_client.search(
        query=state["customer_query"],
        agent_id="insurance_semantic_agent",
        limit=5
    )
    
    state["semantic_knowledge"] = semantic_memories
    return state

## Response Generation with Semantic Memory

Here's where domain expertise meets customer questions. The process combines:

**Input Integration:**
- Customer's specific question
- Retrieved semantic knowledge from past conversations
- General insurance principles learned over time

This demonstrates how AI agents evolve from basic chatbots to knowledgeable specialists through semantic learning.

In [9]:
async def generate_response(state: SupportState) -> SupportState:
    """Generate response using semantic knowledge"""
    
    # Extract semantic insights
    semantic_insights = []
    for memory in state["semantic_knowledge"]:
        content = memory.get('memory', '')
        if content:
            semantic_insights.append(content)
    
    semantic_context = "\n".join(semantic_insights) if semantic_insights else "No relevant insurance knowledge found"

    print("Semantic Context:", semantic_context)
    
    # Generate response using semantic knowledge
    prompt = f"""
You are an insurance support agent. Use the general insurance knowledge to answer the customer's question.

Customer Question: {state["customer_query"]}

Relevant Insurance Knowledge:
{semantic_context}

Provide a helpful response that applies the general insurance principles to answer their specific question.
If the semantic knowledge is relevant, explain the concepts clearly.
"""

    response = await llm.ainvoke(prompt)
    state["support_response"] = response.content.strip()
    
    return state

## Response Generation WITHOUT Memory (Baseline Comparison)

This function simulates a basic AI agent without access to semantic memory - essentially what most chatbots are today:

**Limitations:**
- **No Domain Expertise**: Can only rely on general training data
- **Inconsistent Responses**: May give different answers to similar questions
- **Generic Guidance**: Cannot provide specific insurance principles or industry knowledge
- **No Learning**: Each conversation starts from scratch


In [10]:
async def generate_response_without_memory(query: str) -> str:
    """Generate response WITHOUT semantic memory - simulating a basic agent"""
    
    prompt = f"""
You are an insurance support agent. Answer the customer's question based only on general common sense.
You do NOT have access to specific insurance knowledge or precedents.

Customer Question: {query}

Provide your best guess answer, but you cannot reference specific insurance rules or concepts you're not certain about.
"""

    response = await llm.ainvoke(prompt)
    return response.content.strip()

## Continuous Learning: Storing New Knowledge

This function demonstrates how semantic memory systems continuously improve by learning from every interaction:

**The Compound Effect:**
Each customer interaction makes the entire system smarter. The 100th customer benefits from insights learned from the first 99 conversations, creating increasingly sophisticated and helpful responses.

In [17]:
async def store_new_knowledge(state: SupportState) -> SupportState:
    """Store new semantic knowledge from this interaction"""
    
    # Store the Q&A as potential semantic knowledge
    interaction = f"Q: {state['customer_query']} A: {state['support_response']}"
    
    memory_client.add(
        messages=[{"role": "assistant", "content": interaction}],
        agent_id="insurance_semantic_agent",
        metadata={
            "type": "semantic_knowledge",
        }
    )
    
    return state

## Workflow Orchestration: Bringing It All Together

The `StateGraph` orchestrates our semantic memory workflow, creating a systematic process for handling customer queries:

**The Workflow Steps:**
1. **Retrieve Semantic Knowledge**: Search accumulated domain expertise
2. **Generate Response**: Combine customer query with relevant knowledge  
3. **Store New Knowledge**: Learn from this interaction for future use

In [18]:
def create_semantic_workflow() -> StateGraph:
    """Create simplified workflow for semantic memory demo"""
    
    workflow = StateGraph(SupportState)
    
    # Add nodes
    workflow.add_node("retrieve_semantic_knowledge", retrieve_semantic_knowledge)
    workflow.add_node("generate_response", generate_response)
    workflow.add_node("store_new_knowledge", store_new_knowledge)
    
    # Define flow
    workflow.set_entry_point("retrieve_semantic_knowledge")
    workflow.add_edge("retrieve_semantic_knowledge", "generate_response")
    workflow.add_edge("generate_response", "store_new_knowledge")
    workflow.add_edge("store_new_knowledge", END)
    
    # Compile
    memory_saver = MemorySaver()
    return workflow.compile(checkpointer=memory_saver)

## Workflow Compilation

The workflow is now compiled and ready to handle customer queries. The `MemorySaver` checkpointer ensures that conversation states are maintained across interactions, enabling stateful processing of customer requests.

In [19]:
graph = create_semantic_workflow()

## Storing sample conversations for the demo

This function stores our first training conversation - a detailed discussion about tree damage liability that will become the foundation of our insurance knowledge base.

**Key Concepts That Will Be Extracted:**
- Acts of God principles for natural events
- Property coverage vs. liability coverage distinctions  
- Negligence requirements and documentation
- Tree damage liability rules

Once stored, Mem0 will automatically extract these principles and make them available for future customer interactions, even with completely different scenarios.

In [12]:
def store_semantic_conversations():
    """Store the semantic memory examples from earlier"""
    
    # First conversation - Learning about tree damage and liability
    conversation1 = {
        "conversation_id": "SEM-001",
        "customer_id": "CUST-23456",
        "metadata": {
            "type": "coverage_education",
            "insurance_type": "home"
        },
        "conversation": [
            {
                "role": "user",
                "content": "My neighbor's tree looks diseased and is leaning toward my house. If it falls and damages my roof, whose insurance covers it?"
            },
            {
                "role": "assistant",
                "content": "Great question! This is a common concern. If the tree falls due to natural causes like wind, disease, or storms, YOUR homeowner's insurance would cover the damage to your house, regardless of whose tree it is. This falls under your property coverage with your deductible applying."
            },
            {
                "role": "user",
                "content": "Really? Even though it's their tree? That doesn't seem fair."
            },
            {
                "role": "assistant",
                "content": "I understand it seems counterintuitive, but this is standard across all insurance companies. It's based on the legal principle that acts of nature are 'Acts of God' - no one's at fault. However, there's an important exception: if you formally notify your neighbor in writing that their tree is dangerous and they ignore it, then they could be liable for negligence."
            },
            {
                "role": "user",
                "content": "So I should document this? What exactly should I do?"
            },
            {
                "role": "assistant",
                "content": "Yes! Take photos of the diseased tree, especially showing the lean toward your property. Send your neighbor a certified letter describing the hazard. Keep copies of everything. If they don't act and the tree later falls, their liability insurance would likely cover your damage instead. This documentation shifts it from an 'Act of God' to negligence."
            },
            {
                "role": "user",
                "content": "That's really helpful. What's the difference between my property coverage and their liability coverage in this case?"
            },
            {
                "role": "assistant",
                "content": "Property coverage repairs physical damage to your structures - it's first-party coverage, meaning it covers YOUR property. Liability coverage is third-party - it pays when you're legally responsible for damage to OTHERS. Without proof of negligence, tree damage defaults to property coverage. With negligence, it becomes a liability issue for the tree owner."
            }
        ]
    }
    
    # Store the conversation - mem0 will extract semantic insights
    memory_client.add(
        messages=conversation1["conversation"],
        user_id=conversation1["customer_id"],
        agent_id="insurance_semantic_agent",
        metadata=conversation1["metadata"]
    )

In [ ]:
# store_semantic_conversations()

In [14]:
async def handle_query(customer_id: str, query: str) -> Dict[str, Any]:
    """Process a customer query using semantic memory"""
    
    initial_state = SupportState(
        customer_id=customer_id,
        customer_query=query,
        semantic_knowledge=[],
        support_response=""
    )
    
    config = {"configurable": {"thread_id": f"semantic_{customer_id}"}}
    result = await graph.ainvoke(initial_state, config)
    
    return {
        "query": query,
        "response": result["support_response"],
        "semantic_knowledge_used": len(result["semantic_knowledge"])
    }

## It's time put this agent to the test

In [21]:
test_query1 = "A storm knocked my neighbor's tree onto my garage. They say it's my insurance problem. Who's right and why?"

print("\n❌ WITHOUT SEMANTIC MEMORY (Basic Agent):")
print("-" * 60)
response_without_memory1 = await generate_response_without_memory(test_query1)
print(f"Customer: {test_query1}")
print(f"\nAgent: {response_without_memory1}")
print("\n⚠️  Notice: Vague response, no specific insurance knowledge, unhelpful")



❌ WITHOUT SEMANTIC MEMORY (Basic Agent):
------------------------------------------------------------
Customer: A storm knocked my neighbor's tree onto my garage. They say it's my insurance problem. Who's right and why?

Agent: Based on common sense, this seems like a situation where your neighbor might not be entirely correct. If the tree fell due to a natural storm and wasn't something your neighbor could have prevented, it seems like this could be more of an "act of nature" scenario. 

My general reasoning would be:
- The storm caused the damage, not your neighbor's intentional actions
- Trees can fall during severe weather through no one's fault
- Your own insurance might be the most straightforward way to get repairs done quickly

However, I'd recommend:
1. Talking directly with your neighbor
2. Checking with your insurance company
3. Possibly getting an expert opinion about the tree's condition before the storm

The key is to stay calm and collaborative, as these situations can 

In [25]:
print("\n✅ WITH SEMANTIC MEMORY (Knowledgeable Agent):")
print("-" * 60)
result_with_memory1 = await handle_query(
    customer_id="CUST-98765",
    query=test_query1
)
print(f"Customer: {test_query1}")
print(f"\nAgent: {result_with_memory1['response']}")
print(f"\n✨ Semantic knowledge pieces used: {result_with_memory1['semantic_knowledge_used']}")


✅ WITH SEMANTIC MEMORY (Knowledgeable Agent):
------------------------------------------------------------
Semantic Context: A tree falling due to a storm is considered an 'Act of God' and is covered by the homeowner's property insurance of the property owner where the damage occurred
Acts of nature like wind, disease, or storms causing tree damage are considered 'Acts of God' and covered by the homeowner's property insurance
If a neighbor's tree branches fall and damage your property, your homeowner's insurance will likely cover the damage under property damage coverage as a first-party claim
If you provide written notice to your neighbor about hazardous tree branches and they fail to act, their liability insurance might be responsible for damages due to negligence
Neighbor is not automatically responsible for damage caused by a healthy tree falling in a storm without prior negligence
Hail storm damage is considered an 'Act of God' and thus covered by property insurance
Property dama

In [26]:
test_query2 = "What's the difference between property and liability coverage? I'm confused about which one covers what."

print("\n❌ WITHOUT SEMANTIC MEMORY (Basic Agent):")
print("-" * 60)
response_without_memory2 = await generate_response_without_memory(test_query2)
print(f"Customer: {test_query2}")
print(f"\nAgent: {response_without_memory2}")
print("\n⚠️  Notice: Generic explanation, lacks precision and examples")


❌ WITHOUT SEMANTIC MEMORY (Basic Agent):
------------------------------------------------------------
Customer: What's the difference between property and liability coverage? I'm confused about which one covers what.

Agent: Based on common sense, here's my understanding:

Property coverage seems like it would protect the physical things you own - like your house, car, or personal belongings. So if something gets damaged or destroyed, this type of coverage might help you replace or repair those items.

Liability coverage sounds like it's more about protecting you if you accidentally cause damage or injury to someone else. For example, if someone gets hurt on your property or if you accidentally damage someone else's stuff, liability coverage might help cover the costs of legal issues or compensation.

Think of property coverage as protecting your own things, and liability coverage as protecting you from potential financial risks if you accidentally cause problems for others.

But I wa

In [27]:
print("\n\n✅ WITH SEMANTIC MEMORY (Knowledgeable Agent):")
print("-" * 60)
result_with_memory2 = await handle_query(
    customer_id="CUST-87654",
    query=test_query2
)
print(f"Customer: {test_query2}")
print(f"\nAgent: {result_with_memory2['response']}")
print(f"\n✨ Semantic knowledge pieces used: {result_with_memory2['semantic_knowledge_used']}")




✅ WITH SEMANTIC MEMORY (Knowledgeable Agent):
------------------------------------------------------------
Semantic Context: Liability coverage is third-party coverage and pays when you are legally responsible for damage to others
Property damage coverage in homeowner's insurance covers damage to the insured's property caused by natural events, regardless of tree ownership.
Property coverage repairs physical damage to your structures and is first-party coverage
Hail damage to a roof is covered under property coverage, not liability coverage
Property coverage protects the policyholder's own property from natural events like storms and hail
The property owner typically pays the policy deductible, and the insurance company manages the rest of the claim process
Homeowner's property damage coverage handles repair or replacement costs for damage caused by natural events like storms
If a neighbor's tree branches fall and damage your property, your homeowner's insurance will likely cover the